### Shot-Detection

In [ ]:
# def detect_scenes(video_path, scene_list):
#     cap = cv2.VideoCapture(video_path)   
#     embeddings = []
#     metadatas = []
#     ids = []
#     with tqdm(total = len(scene_list), desc = "Processing frames") as pbar:
#         for i, scene in enumerate(scene_list):
#             start_time, end_time = scene[0].get_seconds(), scene[1].get_seconds()
#             mid_time = (start_time + end_time) / 2
#             timestamps = mid_time
#             labels = "middle"
            
            
#             cap.set(cv2.CAP_PROP_POS_MSEC, timestamps * 1000)
#             ret, frame = cap.read()
            
#             if ret:
#                 img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#                 image = Image.fromarray(img_rgb)
#                 inputs = processor(images = image, return_tensors = 'pt').to(device)
#                 blip_input = blip_processor(images = image, return_tensors = 'pt').to(device)
#                 with torch.no_grad():
#                     outputs = model.get_image_features(inputs.pixel_values)
#                     blip_outputs = blip_model.generate(**blip_input,
#                                                     # max_length = 60,
#                                                     # min_length = 20,
#                                                     # no_repeat_ngram_size=2,
#                                                     # num_beams = 5,
#                                                     )
                
#                 caption = blip_processor.decode(blip_outputs[0], skip_special_tokens=True)
#                 image_embedding = outputs.pooler_output
#                 image_embedding = image_embedding / image_embedding.norm(dim = -1, keepdim= True)
#                 image_embedding = image_embedding.squeeze(0).cpu().numpy().tolist()
#                 timestamp_sec = timestamps*1000
#                 frame_id = f"{video_path}:{timestamp_sec}"
            
#                 ids.append(frame_id)
#                 embeddings.append(image_embedding)

#                 metadatas.append({
#                     "frame_idx": f"frame_no_{i}_{labels}",
#                     "caption": caption,
#                     "timestamp_ms": timestamp_sec,
#                     "source_path": video_path
#                 })
#             pbar.update(1)
                
#     return embeddings, metadatas, ids

In [2]:
!pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/40.2 MB ? eta -:--:--
   -- ------------------------------------- 2.1/40.2 MB 16.3 MB/s eta 0:00:03
   ---- ----------------------------------- 4.7/40.2 MB 14.3 MB/s eta 0:00:03
   ------- -------------------------------- 7.6/40.2 MB 13.9 MB/s eta 0:00:03
   ---------- ----------------------------- 10.2/40.2 MB 13.8 MB/s eta 0:00:03
   ------------- -------------------------- 13.1/40.2 MB 13.7 MB/s eta 0:00:02
   --------------- ------------------------ 15.7/40.2 MB 13.6 MB/s eta 0:00:02
   ------------------ --------------------- 18.4/40.2 MB 13.6 MB/s eta 0:00:02
   --------------------- ------------------ 21.2/40.2 MB 13.6 MB/s eta 0:00:02
   ----------------------- ---------------- 23.9/40.2 MB 13.5 MB/s eta 0:00:02
   -------------------------- ------------- 26.5/40.2 MB 13.5 MB/s eta 0:00:02
   ---------------------------- ----------- 29.1/40.2 MB 13.5 MB/


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: C:\Python314\python.exe -m pip install --upgrade pip


In [1]:
import sys
print(sys.executable)


c:\Users\fareh\coding\video-rag\.venv\Scripts\python.exe


In [2]:
import cv2
from scenedetect import open_video, SceneManager
import scenedetect
from scenedetect.detectors import ContentDetector
import os
from PIL import Image
from transformers import CLIPProcessor, CLIPModel, BlipProcessor, BlipForConditionalGeneration
import torch
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32",use_safetensors=True).to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to(device)

c:\Users\fareh\coding\video-rag\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\fareh\coding\video-rag\.venv\Lib\site-packages\moviepy\video\io\ffmpeg_reader.py:294: SyntaxWarning: invalid escape sequence '\d'
  lines_video = [l for l in lines if ' Video: ' in l and re.search('\d+x\d+', l)]
c:\Users\fareh\coding\video-rag\.venv\Lib\site-packages\moviepy\video\io\ffmpeg_reader.py:367: SyntaxWarning: invalid escape sequence '\d'
  rotation_lines = [l for l in lines if 'rotate          :' in l and re.search('\d+$', l)]
c:\Users\fareh\coding\video-rag\.venv\Lib\site-packages\moviepy\video\io\ffmpeg_reader.py:370: SyntaxWarning: invalid escape sequence '\d'
  match = re.search('\d+$', rotation_line)
c:\Users\fareh\coding\video-rag\.venv\Lib\site-packages\moviepy\config_defaults.py:47: SyntaxWarning: inva

In [3]:
def scene_split(video_path):
    print("--- Detecting shot boundaries with PySceneDetect ---")
    video = open_video(video_path)
    scene_manager = SceneManager()
    scene_manager.add_detector(ContentDetector())

    try:
        scene_manager.detect_scenes(video, show_progress=False)
        scene_list = scene_manager.get_scene_list()
    except Exception as e:
        print("Scene detection failed:", e)
        scene_list = []
    return scene_list

In [7]:

from ollama import chat


def generate_caption(frame,buffer):
    frame.save(buffer, format="JPEG")
    img_bytes = buffer.getvalue()
    response = chat(
    model='gemma3',
    messages=[
        {
    "role": "system",
    "content": "You are a helpful assistant that can analyze images and provide captions."
    },

    {
        'role': 'user',
        'content': 'What is in this image? .',
        'images': [img_bytes],
    }
    ],
    )

    return response.message.content

In [10]:

from io import BytesIO


def detect_scenes(video_path, scene_list):
    cap = cv2.VideoCapture(video_path)   
    embeddings = []
    metadatas = []
    ids = []
    buffer = BytesIO()
    with tqdm(total = len(scene_list), desc = "Processing frames") as pbar:
        for i, scene in enumerate(scene_list):
            start_time, end_time = scene[0].get_seconds(), scene[1].get_seconds()
            mid_time = (start_time + end_time) / 2
            timestamps = [start_time, mid_time, end_time]
            labels = ["initial", "middle", "final"]
            
            for t, label in zip(timestamps, labels):
                cap.set(cv2.CAP_PROP_POS_MSEC, t * 1000)
                ret, frame = cap.read()
                
                if ret:
                    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    image = Image.fromarray(img_rgb)
                    inputs = processor(images = image, return_tensors = 'pt').to(device)
                    caption=generate_caption(image,buffer)
                    #print(caption)
                    # blip_input = blip_processor(images = image, return_tensors = 'pt').to(device)
                    with torch.no_grad():
                        outputs = model.get_image_features(inputs.pixel_values)
                        # blip_outputs = blip_model.generate(**blip_input,
                        #                                 # max_length = 500,
                        #                                 # min_length = 150,
                        #                                 # no_repeat_ngram_size=2,
                        #                                 # num_beams = 5,
                        #                                 )
                    
                    # caption = blip_processor.decode(blip_outputs[0], skip_special_tokens=True)
                    image_embedding = outputs.pooler_output
                    image_embedding = image_embedding / image_embedding.norm(dim = -1, keepdim= True)
                    image_embedding = image_embedding.squeeze(0).cpu().numpy().tolist()
                    timestamp_sec = t*1000
                    frame_id = f"{video_path}:{timestamp_sec}"
                
                    ids.append(frame_id)
                    embeddings.append(image_embedding)

                    metadatas.append({
                        "frame_idx": f"frame_no_{i}_{label}",
                        "caption": caption,
                        "timestamp_ms": timestamp_sec,
                        "source_path": video_path
                    })
            pbar.update(1)
                
    return embeddings, metadatas, ids

In [5]:
scene_list = scene_split("../This Integral Breaks Math.mp4")

--- Detecting shot boundaries with PySceneDetect ---


In [14]:
emb, met, ids = detect_scenes(video_path="../This Integral Breaks Math.mp4", scene_list= scene_list)

Processing frames:  12%|█▏        | 6/49 [01:33<11:09, 15.56s/it]


KeyboardInterrupt: 

In [7]:
import chromadb
client = chromadb.PersistentClient(path = "../db_path")
collection = client.get_or_create_collection("frame_collection")

In [8]:
collection.add(
    ids = ids,
    embeddings= emb,
    metadatas= met,
)

In [13]:
w[0]

(00:00:00.000 [frame=0, fps=30.000], 00:00:01.233 [frame=37, fps=30.000])

In [13]:
# Load model directly
from transformers import AutoProcessor, AutoModelForImageTextToText

processor = AutoProcessor.from_pretrained("google/gemma-3-4b-it")
model = AutoModelForImageTextToText.from_pretrained("google/gemma-3-4b-it")
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "url": "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/p-blog/candy.JPG"},
            {"type": "text", "text": "What animal is on the candy?"}
        ]
    },
]
inputs = processor.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=40)
print(processor.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/google/gemma-3-4b-it.
401 Client Error. (Request ID: Root=1-6988df00-0c02343335eec9b9385a4499;82209a26-925c-4ae7-b207-ab86528e1a8e)

Cannot access gated repo for url https://huggingface.co/google/gemma-3-4b-it/resolve/main/config.json.
Access to model google/gemma-3-4b-it is restricted. You must have access to it and be authenticated to access it. Please log in.